<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/datasets/Churn.csv')
display(df.describe())
print(df.info())
display(df.corr())
display(df.head())

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.007322,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.021418,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.007322,-0.021418,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Пустые значения tenure скорее всего указывают на клиентов, являющимися клиентами банка меньше года. Заменим пустые значения на 0. Сильной корреляции между фичами нет, удалять ничего не мужно. Первые три колонки смысловой нагрузки не несут, их из датафрейма можно будет удалить. Преобразуем пол в числовые столбцы методом OHE. Если кол-во стран будет небольшим, их также преобразуем в числовые столбцы. Учитывая разброс стандартных отклонений в фичах, имеет смысл произвести их масштабирование.

In [2]:
print(df['Geography'].value_counts())

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64


Действительно, страны всего 3, можно смело преобразовывать. Также заменим пустые значения числа лет в банке на 0.

In [3]:
df = pd.get_dummies(data = df, columns = ['Geography', 'Gender'])
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1 )
df['Tenure'] = df['Tenure'].fillna(0)

In [4]:
display(df)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,0,1,0,0,1


## Исследование задачи

In [5]:
print(df['Exited'].value_counts(normalize = True))

0    0.7963
1    0.2037
Name: Exited, dtype: float64


Дизбаланс классов есть. В П.3 исправим.

Разобьем данные на выборки (train/valid/test - 60%/20%/20%).

In [6]:
features = df.drop('Exited', axis = 1)
target = df['Exited']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)
print('Размер тренировочной выборки:', features_train.shape,target_train.shape)
print('Размер тестовой выборки:',features_test.shape,target_test.shape)
print('Размер валидационной выборки:',features_valid.shape,target_valid.shape)


Размер тренировочной выборки: (6000, 13) (6000,)
Размер тестовой выборки: (2000, 13) (2000,)
Размер валидационной выборки: (2000, 13) (2000,)


Попробуем построить модели

In [7]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('DecisionTreeClassifier f1 - ', f1_score(target_valid , predicted_valid))

model = RandomForestClassifier(2)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('RandomForestClassifier f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('LogisticRegression f1 - ', f1_score(target_valid , predicted_valid))

DecisionTreeClassifier f1 -  0.47158403869407495
RandomForestClassifier f1 -  0.3771043771043771
LogisticRegression f1 -  0.13441955193482688


Результаты не впечатляющие. Произведем масштабирование фичей. Возможно, это улучшит F1.

In [8]:
features_train_scaled = features_train.copy()
features_valid_scaled = features_valid.copy()


numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train_scaled[numeric])
features_train_scaled[numeric] = scaler.transform(features_train_scaled[numeric])
features_valid_scaled[numeric] = scaler.transform(features_valid_scaled[numeric])

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)
print('DecisionTreeClassifier f1 - ', f1_score(target_valid , predicted_valid))

model = RandomForestClassifier(random_state=12345, n_estimators = 100)
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)
print('RandomForestClassifier f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)
print('LogisticRegression f1 - ', f1_score(target_valid , predicted_valid))

DecisionTreeClassifier f1 -  0.47030303030303033
RandomForestClassifier f1 -  0.5293233082706768
LogisticRegression f1 -  0.2743055555555555


Улучшение наблюдается только для логистической регрессии.

В итоге лучшей оказалась модель случайного леса с масштабированными фичами. F1 для неё достиг 0.529.
На втором месте дерево решений без масштабированных фичей, F1 составил 0.47.
Показатели этих дву моделей почти не изменились после масштабирования фичей, зато для логистической регрессии рост оказался более чем двухкратным: с 0.13 до 0.27.

## Борьба с дисбалансом

Для начала попробуем для каждой из моделей взвесить классы. Эта мера кажется оптимальной, так как не придется изменять семплы. Для модели логистической регрессии взвесим классы с учетом и без учета масштабирования фичей.

In [9]:

model = DecisionTreeClassifier(random_state=12345, class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('DecisionTreeClassifier with weighted classes AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('DecisionTreeClassifier with weighted classes f1 - ', f1_score(target_valid , predicted_valid))

model = RandomForestClassifier(random_state=12345, n_estimators = 100, class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier with weighted classes AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('RandomForestClassifier with weighted classes f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear', class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression with weighted classes AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression with weighted classes f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear', class_weight = 'balanced')
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression scaled with weighted classes AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression scaled with weighted classes f1 - ', f1_score(target_valid , predicted_valid))

DecisionTreeClassifier with weighted classes AUC-ROC -  0.6724471607969768
DecisionTreeClassifier with weighted classes f1 -  0.48536585365853663
RandomForestClassifier with weighted classes AUC-ROC -  0.8490040790260707
RandomForestClassifier with weighted classes f1 -  0.522273425499232
LogisticRegression with weighted classes AUC-ROC -  0.7327390937396469
LogisticRegression with weighted classes f1 -  0.4630705394190871
LogisticRegression scaled with weighted classes AUC-ROC -  0.5001648999881572
LogisticRegression scaled with weighted classes f1 -  0.4784110535405872


В лучшую сторону улучшились показатели модели дерева решений (примерно +0.02), и логистической регресии (+0.2). Интересно, что при взвешенных классах масштабирование признаков не играет роли - для двух моделей результаты получились идентичными. Для модели случайного леса при 100 деревьев результаты при взвешенных классах оказались хуже на 0.02, чем при невзвешенных.

AUC-ROC по моделям в целом пропорционален F1. Наилучший у модели случайного леса, почти 0.84, что заметно лучше рандомной модели. Однако для логистической регрессии с взвешенными классами AUC-ROC почти такая же, как для случайной модели.

Попробуем ребалансировать классы через upsampling. Так классов со значением 1 меньше 0 примерно в 4 раза, на это число и умножим при ребалансировке.


In [10]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
print('target values amount after upsampling: \n', target_upsampled.value_counts())

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('DecisionTreeClassifier upsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('DecisionTreeClassifier upsampled f1 - ', f1_score(target_valid , predicted_valid))

model = RandomForestClassifier(random_state=12345, n_estimators = 100 )
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier upsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('RandomForestClassifier upsampled f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression upsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression upsampled f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid_scaled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression scaled features upsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression scaled features upsampled f1 - ', f1_score(target_valid , predicted_valid))

target values amount after upsampling: 
 0    4804
1    4784
Name: Exited, dtype: int64
DecisionTreeClassifier upsampled AUC-ROC -  0.6752144824164146
DecisionTreeClassifier upsampled f1 -  0.4901960784313726
RandomForestClassifier upsampled AUC-ROC -  0.8434447007889715
RandomForestClassifier upsampled f1 -  0.5863874345549738
LogisticRegression upsampled AUC-ROC -  0.6854442780453655
LogisticRegression upsampled f1 -  0.432
LogisticRegression scaled features upsampled AUC-ROC -  0.6854442780453655
LogisticRegression scaled features upsampled f1 -  0.4690265486725663


У дерева решений и логистической регресии значения меняются в пределах 0.1, но зато какой внушительный скачок у модели случайного леса - F1 почти 0.59! Можно будет немного подкрутить гиперпараметры и добиться более высоких значений.

Впечатляющий рост на апсемплинге показал показатель AUC-ROC, особенно на моделях логистической регрессии. Теперь, с показателем, начинающимся от 0.67, можно утверждать, что все модели в этом плане лучше случайной.

Попробуем downsampling.

In [11]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

print('target values amount after downsampling: \n', target_downsampled.value_counts())

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('DecisionTreeClassifier downsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('DecisionTreeClassifier downsampled f1 - ', f1_score(target_valid , predicted_valid))

model = RandomForestClassifier(random_state=12345, n_estimators = 100 )
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('RandomForestClassifier downsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('RandomForestClassifier downsampled f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression downsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression downsampled f1 - ', f1_score(target_valid , predicted_valid))

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid_scaled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('LogisticRegression scaled features downsampled AUC-ROC - ',roc_auc_score(target_valid, probabilities_one_valid))
print('LogisticRegression scaled features downsampled classes f1 - ', f1_score(target_valid , predicted_valid))

target values amount after downsampling: 
 0    1201
1    1196
Name: Exited, dtype: int64
DecisionTreeClassifier downsampled AUC-ROC -  0.697749564888895
DecisionTreeClassifier downsampled f1 -  0.49612403100775193
RandomForestClassifier downsampled AUC-ROC -  0.8426996526606614
RandomForestClassifier downsampled f1 -  0.5708989805375349
LogisticRegression downsampled AUC-ROC -  0.6842315135870096
LogisticRegression downsampled f1 -  0.4285714285714286
LogisticRegression scaled features downsampled AUC-ROC -  0.6842315135870096
LogisticRegression scaled features downsampled classes f1 -  0.46893317702227433


В целом этот метод показал себя хорошо, если смотреть по трём моделям. В модели дерева решений F1 равен почти 0.5, в модели случайного леса показатель превысил 0.57, в логистической регрессии более 0.42 и почти 0.47 в случае с масштабированными фичами.

Downsampling показал наилучшие показатели AUC-ROC по всем построенным моделям - от 0.68 до 0.84.

Попробуем выжать максимум из моделей случайного леса, перебрав гиперпараметры. Используем все три метода ребалансировки классов.

In [12]:
#%%time
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for est in range(100, 150, 10):
    for depth in range(7,12):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth,class_weight = 'balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid , predicted_valid) 
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print("F1 наилучшей модели random forest, взвешивание классов:", best_forest_result,
      '\nГлубина дерева:',  best_forest_depth,
      '\nКоличество деревьев:', est
     )

F1 наилучшей модели random forest, взвешивание классов: 0.6181410974244121 
Глубина дерева: 9 
Количество деревьев: 140


In [13]:
#%%time
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for est in range(100, 150, 10):
    for depth in range(7,12):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid , predicted_valid) 
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print("F1 наилучшей модели random forest, upsampling классов:", best_forest_result,
      '\nГлубина дерева:',  best_forest_depth,
      '\nКоличество деревьев:', est
     )

F1 наилучшей модели random forest, upsampling классов: 0.611764705882353 
Глубина дерева: 10 
Количество деревьев: 140


In [14]:
#%%time
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for est in range(100, 150, 10):
    for depth in range(7,12):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid , predicted_valid) 
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print("F1 наилучшей модели random forest, downsampling классов:", best_forest_result,
      '\nГлубина дерева:',  best_forest_depth,
      '\nКоличество деревьев:', est
     )

F1 наилучшей модели random forest, downsampling классов: 0.585635359116022 
Глубина дерева: 11 
Количество деревьев: 140


Upsampling и взвешивание классов при оптимально подобранных параметрах показывают примерно одинаковое значение F1: чуть выше 0.61. На тестовых данных попробуем модель со взвешенными классами, глубиной дерева 9 и 140 деревьями.

В итоге наиболее чувствительным к изменению гиперпараметров для модели random forest оказался метод взвешивания классов. Для него удалось достичь как наибольшего скачка F1, так и его наибольшего значения в принципе. Upsampling показал относительно высокие результаты на всех моделях до и после подбора гиперпараметров. Downsampling, несмотря на эффективность на других моделях, на random forest при оптимальных гиперпараметрах показал себя хуже остальных.

## Тестирование модели

In [15]:
model = RandomForestClassifier(random_state=12345, n_estimators=140, max_depth = 9, class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('AUC-ROC на тестовой выборке, random forest, ребалансировка классов:',roc_auc_score(target_test, probabilities_one_test))
print('F1 на тестовой выборке, random forest, ребалансировка классов:', f1_score(target_test , predicted_test)) 

AUC-ROC на тестовой выборке, random forest, ребалансировка классов: 0.855329695921219
F1 на тестовой выборке, random forest, ребалансировка классов: 0.6185101580135439


На тестовой выборке результат оказался даже чуть лучше - почти 0.63, а AUC-ROC более 0.85. Удачно засемплировали!

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=140, max_depth = 10)
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('AUC-ROC на тестовой выборке, random forest, upsampling',roc_auc_score(target_test, probabilities_one_test))
print('F1 на тестовой выборке, random forest, upsampling:', f1_score(target_test , predicted_test)) 

AUC-ROC на тестовой выборке, random forest, upsampling 0.8495061063761578
F1 на тестовой выборке, random forest, upsampling: 0.621978021978022


Чуть хуже результат при технике upsampling - F1 немногим выше 0.62, а AUC-ROC немного не дотягивает до 0.85.

<b>Выводы</b>

Балансирование классов оказало существенное положительное влияние на прогнозирующую способность всех построенных моделей. Лучшей оказалась модель случайного леса со 140 деревьями и глубиной 9, где классы были взвешены. Показатель F1 на тестовой выборке в ней превысил 0.629, а ROC-AUC превысил 0.85, что указывает на эффективность модели по сравнению со случайной.